## `LSTM`
***
***
Time: 2020-09-14<br>
Author: dsy
***

![LSTM](./imgs/LSTM.png)

$$
\begin{aligned}
c_t &= f_t \odot c_{t-1} + i_t \odot \tilde{c_t} \\
h_t &= \omicron_t \odot \tanh(c_t)\\
\tilde{c_t} &= \tanh(W_c x_t + U_c h_{t-1} + b_c)\\
i_t &= \sigma(W_ix_t + U_ih_{t-1}+b_i) \\
f_t &= \sigma(W_fx_t + U_fh_{t-1}+b_f) \\
\omicron_t &= \sigma(W_\omicron x_t+U_\omicron h_{t-1} + b_\omicron)
\end{aligned}
$$

In [1]:
import torch
import torch.nn as nn

In [2]:
seed = 0
torch.manual_seed(seed)

In [3]:
class LSTMReal(nn.Module):
    def __init__(self,hidden_size: int, bias: bool = True):
        super(LSTMReal,self).__init__()
        self.hidden_size = hidden_size
        self.bias = bias
        
        
#     def lstm_cell(self,input, hidden, w_ih, w_hh, b_ih, b_hh):
#         # type: (Tensor, Tuple[Tensor, Tensor], Tensor, Tensor, Tensor, Tensor) -> Tuple[Tensor, Tensor]
#         hx, cx = hidden


#         return hy, cy
        
    def forward(self,input):
        batch, input_size = input.size()
        
        zeros = torch.zeros(input.size(0), self.hidden_size, dtype=input.dtype, device=input.device)
        hx,cx = zeros, zeros
        
        weight_ih = nn.parameter.Parameter(torch.Tensor( 4 * self.hidden_size, input_size))
        weight_hh = nn.parameter.Parameter(torch.Tensor( 4 * self.hidden_size, self.hidden_size))
        bias_ih = nn.parameter.Parameter(torch.Tensor( 4 * self.hidden_size))
        bias_hh = nn.parameter.Parameter(torch.Tensor( 4 * self.hidden_size))
        
        import math
        stdv = 1.0 / math.sqrt(self.hidden_size)
#         for weight in self.parameters():
        nn.init.uniform_(weight_ih, -stdv, stdv)
        nn.init.uniform_(weight_hh, -stdv, stdv)
        nn.init.uniform_(bias_ih, -stdv, stdv)
        nn.init.uniform_(bias_hh, -stdv, stdv)
        
        
#         hy,cy = self.lstm_cell(input, hx, weight_ih, weight_hh, bias_ih, bias_hh)
        gates = torch.mm(input, weight_ih.t()) + torch.mm(hx, weight_hh.t()) + bias_ih + bias_hh

        ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)

        ingate = torch.sigmoid(ingate)
        forgetgate = torch.sigmoid(forgetgate)
        cellgate = torch.tanh(cellgate)
        outgate = torch.sigmoid(outgate)

        cy = (forgetgate * cx) + (ingate * cellgate)
        hy = outgate * torch.tanh(cy)
        
        return hy,cy

参考了`Pytorch`中[LSTMCell源码](https://pytorch.org/docs/stable/_modules/torch/nn/modules/rnn.html#LSTMCell)探究,并写出了如上代码。

In [4]:
data = torch.linspace(1,11,10).reshape((5,-1))
data

tensor([[ 1.0000,  2.1111],
        [ 3.2222,  4.3333],
        [ 5.4444,  6.5556],
        [ 7.6667,  8.7778],
        [ 9.8889, 11.0000]])

In [5]:
lstmreal = LSTMReal(hidden_size = 8, bias = True)
lstmreal(data)

(tensor([[-0.1353, -0.1595,  0.1423,  0.2746, -0.1335, -0.1357, -0.1085,  0.1826],
         [-0.2076, -0.1191,  0.1999,  0.3700, -0.1303, -0.0684, -0.0743,  0.2237],
         [-0.1893, -0.0419,  0.2443,  0.4175, -0.1064, -0.0255, -0.0272,  0.1831],
         [-0.1532, -0.0129,  0.2737,  0.4463, -0.0789, -0.0082, -0.0085,  0.1373],
         [-0.1176, -0.0038,  0.2895,  0.4667, -0.0554, -0.0024, -0.0025,  0.0987]],
        grad_fn=<MulBackward0>),
 tensor([[-0.2940, -0.1949,  0.2190,  0.5302, -0.3503, -0.4094, -0.1767,  0.2899],
         [-0.6095, -0.1266,  0.2743,  0.7525, -0.4790, -0.4205, -0.0978,  0.3964],
         [-0.7520, -0.0426,  0.3082,  0.8667, -0.5697, -0.3566, -0.0316,  0.3547],
         [-0.8305, -0.0130,  0.3248,  0.9270, -0.6335, -0.2789, -0.0091,  0.2937],
         [-0.8829, -0.0038,  0.3288,  0.9599, -0.6808, -0.2086, -0.0026,  0.2368]],
        grad_fn=<AddBackward0>))

In [6]:
lstmcell = torch.nn.LSTMCell(input_size=2, hidden_size=8, bias = True)
lstmcell(data)

(tensor([[ 1.2519e-01,  1.3784e-01, -7.4164e-02,  6.0462e-02, -4.6864e-02,
          -8.8378e-02,  8.5501e-02, -1.5914e-01],
         [-3.4459e-02,  1.3306e-01, -8.9172e-02,  1.1447e-01, -7.5529e-03,
          -2.7284e-01,  8.0205e-02, -2.2535e-01],
         [-3.1308e-01,  7.8082e-02, -7.6456e-02,  1.2036e-01, -8.5775e-04,
          -3.5777e-01,  6.8626e-02, -2.7248e-01],
         [-5.3063e-01,  3.8976e-02, -5.7628e-02,  1.0382e-01, -8.5226e-05,
          -3.9670e-01,  5.2570e-02, -3.0302e-01],
         [-6.4968e-01,  1.8259e-02, -4.0986e-02,  8.1661e-02, -8.0362e-06,
          -4.1630e-01,  3.4095e-02, -3.2113e-01]], grad_fn=<MulBackward0>),
 tensor([[ 0.1952,  0.3762, -0.2585,  0.1158, -0.1784, -0.1611,  0.1761, -0.2441],
         [-0.0435,  0.7112, -0.4460,  0.2637, -0.1034, -0.4538,  0.1731, -0.3642],
         [-0.3691,  0.8793, -0.5621,  0.3411, -0.0504, -0.5437,  0.1553, -0.4661],
         [-0.6384,  0.9509, -0.6301,  0.3675, -0.0225, -0.5527,  0.1248, -0.5489],
         [-0.8098